## 과제

- yes24에서 python을 검색하고 200권의 책의 정보를 수집하는 코드를 작성하고, csv파일과 xlsx파일로 저장하시오  
  (다른 책을 검색해도 됨 / 과제 예시는 python을 검색한 코드로 나갈 것)
- 책사진, 제목, 저자, 가격, 판매지수, 회원리뷰수, 별점 점수, 사이크 링크를 모두 포함해야함
- 책사진은 폴더를 따로 만들어 저장한 후 캡처본으로 제출
- 패키지 이용에는 관여하지 않음
- 제출 목록(4개) : 4차시과제_학번.ipynb파일, 4차시과제_학번.csv파일, 4차시과제_학번.excel파일, 4차시과제_학번.사진저장 폴더의 캡처본

#### Libraries

In [1]:
import requests
from bs4 import BeautifulSoup

import time
from urllib.request import urlretrieve

from selenium import webdriver
from selenium.webdriver.common.keys import Keys # 스크롤 내리기

import openpyxl

# 1. Requests와 BeautifulSoup를 사용한 크롤링

In [2]:
# 검색, 페이지
keyword = 'python'; page =1
# 스크랩한 책의 갯수
total_books = 1
# 제목 / 저자 /가격 / 판매지수 / 회원리뷰수 / 별점 점수 / 링크 리스트
titles = []; writers = []; prices = []; sales_indexs = [];
views = []; rating_scores = []; links = [];


while total_books < 200:
    req = requests.get(f'http://www.yes24.com/searchcorner/Search?keywordAd=&keyword=&domain=BOOK&qdomain=%b1%b9%b3%bb%b5%b5%bc%ad&query={keyword}&PageNumber={page}',headers={'User_Agent':'Mozilla/5.0'})
    print(f'page: {page}, req status: {req.status_code}')

    # html 구조로 인식
    soup = BeautifulSoup(req.text, 'html.parser')
    
    # ----- image 저장
    images = soup.select('td.goods_img')
    for image in images:
        image_src = image.select_one('a img').attrs['src']
        urlretrieve(image_src, f'./product_pictures/beautifulSoup/python/{total_books}.jpg'); total_books +=1
    # ----- 제목 / 저자 /가격 / 판매지수 / 회원리뷰수 / 별점 점수 / 링크 저장
    books = soup.select('td.goods_infogrp')
    for book in books:
        title = book.select_one('p a').text
        titles.append(title.replace(",", "-"))
        if ' 저' in book.select_one('div.goods_info').text:
            writer = book.select_one('div.goods_info').text.split(' 저')[0].strip('\n')
            writers.append(writer.replace(",", "-"))
        elif ' 공저' in book.select_one('div.goods_info').text:
            writer = book.select_one('div.goods_info').text.split(' 공저')[0].strip('\n')
            writers.append(writer.replace(",", "-"))
        elif ' 편저' in book.select_one('div.goods_info').text:
            writer = book.select_one('div.goods_info').text.split(' 편저')[0].strip('\r\n                ')
            writers.append(writer.replace(",", "-"))
        price = book.select_one('div.goods_price em').text
        prices.append(price.replace(",", ""))
        try:
            sales_index = book.select_one('span.gd_reviewCount').text.split()[1]
            sales_indexs.append(sales_index.replace(",", ""))
        except:
            sales_indexs.append('판매지수없음')
        try:
            view = book.select_one('div.goods_rating span a em').text
            views.append(view.replace(",", ""))
        except:
            views.append('0')
        try:
            rating_score = book.select_one('span.gd_rating em').text
            rating_scores.append(rating_score)
        except:
            rating_scores.append('별점미등록')
        href = book.select_one('p a').attrs['href']
        link = 'http://www.yes24.com' + href
        links.append(link)
    page += 1

print('Crawling Finished !')

page: 1, req status: 200
page: 2, req status: 200
page: 3, req status: 200
page: 4, req status: 200
page: 5, req status: 200
page: 6, req status: 200
page: 7, req status: 200
page: 8, req status: 200
page: 9, req status: 200
page: 10, req status: 200
Crawling Finished !


In [3]:
# csv 파일로 저장
f = open('yes24_csv_python.csv', 'w')
f.write('제목, 저자, 가격, 판매지수, 회원리뷰수, 별점 점수, 링크 리스트\n')
for i in range(total_books-1):
    f.write(f'{titles[i]}, {writers[i]}, {prices[i]}, {sales_indexs[i]}, {views[i]}, {rating_scores[i]}, {links[i]}\n')
f.close()

In [4]:
# excel 파일로 저장
wb = openpyxl.Workbook()
sheet = wb.active
sheet.append('제목, 저자, 가격, 판매지수, 회원리뷰수, 별점 점수, 링크 리스트'.split(','))
for i in range(total_books-1):
    sheet.append(f'{titles[i]}, {writers[i]}, {prices[i]}, {sales_indexs[i]}, {views[i]}, {rating_scores[i]}, {links[i]}'.split(','))
wb.save('yes24_excel_python.xlsx')

# 2. Selenium을 사용한 크롤링

In [13]:
# 드라이버 불러오기(크롬으로 빈 페이지가 불러와짐)
dr = webdriver.Chrome('./chromedriver_win32/chromedriver')

# 원하는 페이지 get메소드로 불러오기
dr.get('http://www.yes24.com/')

# 검색입력
search_bar = dr.find_element_by_css_selector('span.schIpt label input')
search_bar.send_keys('python')

# 클릭하기
search = dr.find_element_by_css_selector('span.schBtn button')
search.click()

In [14]:
# 스크랩한 책의 갯수, 페이지
total_books = 1; page = 1
# 제목 / 저자 /가격 / 판매지수 / 회원리뷰수 / 별점 점수 / 링크 리스트
titles = []; writers = []; prices = []; sales_indexs = [];
views = []; rating_scores = []; links = [];

while total_books < 200 :
    # ----- 이미지 저장
    images = dr.find_elements_by_css_selector('td.goods_img')[:20]
    for image in images:
        image_src = image.find_element_by_css_selector('a img').get_attribute('src')
        urlretrieve(image_src, f'./product_pictures/selenium/python/{total_books}.jpg')
        total_books +=1
    # ----- 제목 / 저자 /가격 / 판매지수 / 회원리뷰수 / 별점 점수 / 링크 저장
    books = dr.find_elements_by_css_selector('td.goods_infogrp')[:20]
    for book in books:
        title = book.find_element_by_css_selector('p a').text
        titles.append(title.replace(",","-"))
        writer_infos = book.find_element_by_css_selector('div.goods_info').text
        if ' 저' in writer_infos:
            writer = writer_infos.split(' 저')[0]
            writers.append(writer.replace(",","-"))
        elif ' 공저' in writer_infos:
            writer = writer_infos.split(' 공저')[0]
            writers.append(writer.replace(",","-"))
        elif ' 편저' in writer_infos:
            writer = writer_infos.split(' 편저')[0]
            writers.append(writer)
        elif ' 역' in writer_infos:
            writer = writer_infos.split(' 역')[0]
            writers.append(writer)
        else:
            writer = writer_infos.split(' |')[0]
            writers.append(writer.replace(",","-"))
        price = book.find_element_by_css_selector('div.goods_price em').text.replace(",", "")
        prices.append(price.replace(", ", ""))
        try:
            sales_index = book.find_element_by_css_selector('div.goods_rating > span').text.split()[1]
            sales_indexs.append(sales_index.replace(",",""))
        except:
            sales_indexs.append('판매지수없음')
        try:
            view = book.find_element_by_css_selector('div.goods_rating span a em').text.replace(", ", "")
            views.append(view.replace(",",""))
        except:
            views.append('0')
        try:
            rating_score = book.find_element_by_css_selector('span.gd_rating em').text
            rating_scores.append(rating_score)
        except:
            rating_scores.append('별점미등록')
        link = book.find_element_by_css_selector('p a').get_attribute('href')
        links.append(link)
    print(f'page: {page}')
    page += 1
    try:
        next_page = dr.find_element_by_css_selector(f'a.n:nth-of-type({page+1})')
        next_page.click()
        time.sleep(1.5)
    except:
        break

print('Crawling Finished !')

page: 1
page: 2
page: 3
page: 4
page: 5
page: 6
page: 7
page: 8
page: 9
page: 10
Crawling Finished !


In [15]:
# csv 파일로 저장
f = open('yes24_csv_python_selenium.csv', 'w')
f.write('제목, 저자, 가격, 판매지수, 회원리뷰수, 별점 점수, 링크 리스트\n')
for i in range(total_books-1):
    f.write(f'{titles[i]}, {writers[i]}, {prices[i]}, {sales_indexs[i]}, {views[i]}, {rating_scores[i]}, {links[i]}\n')
f.close()

In [16]:
# excel 파일로 저장
wb = openpyxl.Workbook()
sheet = wb.active
sheet.append('제목, 저자, 가격, 판매지수, 회원리뷰수, 별점 점수, 링크 리스트'.split(','))
for i in range(total_books-1):
    sheet.append(f'{titles[i]}, {writers[i]}, {prices[i]}, {sales_indexs[i]}, {views[i]}, {rating_scores[i]}, {links[i]}'.split(','))
wb.save('yes24_excel_python_selenium.xlsx')

**Fin───────────────────────────────────────────────────────────────────────────────────────────────**